# From Zero to Hero in Collateral Shortfall Monitoring

### Introduction

In this notebook, we will showcase how quickly a dashboard can be put together for a simplified use case of Collateral Shortfall monitoring with Atoti libraries.  
  
Collateral is a form of credit risk mitigation where an asset is accepted as security for extending a loan.  
Market value of a collateral changes over time and lender has to accomodate for it. As such, depending on the amount of risk associated, a percentage of what is known as haircut is applied to the asset's market value. This gives the value of the collateral that can be used for loan, also known as collateral value.   
  
Collateral Shortfall occurs when the collateral value goes below the cash out value. That meant that the value of collateral is less than what it is expected to be, due to a variety of factors such as market fluctuations, contracts enforceability etc. 

We will be creating a multi-dimension data cube and derive the various measures such as market value, collateral value after haircut, cash out value over account and thereafter, the Collateral shortfall for the accounts. 

Leveraging on the data cube and Atoti's data visualization, we will put together dashboards that reflects collateral status of accounts. Cherry on top, we will perform some *What if Analysis* to demonstrate the impact on Collateral in the below scenarios:  

- Haircut simulation where there is a consumption slowdown in the Consumer Cyclical sector in Italy and France
- Technology slump where price value for the sector dipped by 30%
- Cash out simulation where cash out increase by 10%

In [1]:
import atoti as tt
import pandas as pd
import os.path
import wget

In [2]:
# check if file has already been downloaded 
def download_source(cwd: str, url: str, filename: str):
    """This function downloads data file from the given url to the working directory (cwd).

    Args:
        cwd: current working directory.
        url: url for the source file
        filename: filename of the downloaded file
    """
    if os.path.isfile(cwd + "/" + filename):
        print(filename + ' already downloaded.')
    else:
        wget.download(url, out=cwd)

In [3]:
cwd = os.getcwd()
download_source(cwd, 'http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_positions.csv', "assets_positions.csv")
download_source(cwd, 'http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_prices.csv', "assets_prices.csv")
download_source(cwd, 'http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_attributes.parquet', "assets_attributes.parquet")
download_source(cwd, 'http://data.atoti.io/notebooks/collateral-shortfall-monitoring/loans_positions.csv', "loans_positions.csv")

100% [..................................................................................] 142 / 142

In [4]:
# tell atoti to load the database containing the UI dashboards
config = tt.config.create_config(metadata_db="./metadata.db")

# a session has to be created for atoti
session = tt.create_session(config=config)

### Data loading
A session is used to read data of formats csv, parquet, pandas dataframe, numpy and spark.   
Refer to https://www.atoti.io/documentation/tutorial/08-data-sources.html   
   
#### Loading csv

In [5]:
asset_positions_store = session.read_csv('assets_positions.csv', keys=['Account','Asset_Code'], store_name='asset_positions_store')

In [6]:
# use store.head(n) preview the first n rows loaded into the store where needed
# likewise, you can use store.columns to preview the columns in the store
# or store.shape to see the number of rows and columns loaded into the store
asset_positions_store.head(5)

Quantity
Account           Asset_Code          
Niel              CAP.PA      100000.0
                  SAN.PA      100000.0
Musk              ENI.PA      100000.0
                  ENGI.PA     100000.0
Bezos & MacKenzie AC.PA       100000.0

In [7]:
assets_prices = session.read_csv('assets_prices.csv', keys=['Asset_Code','Date'], store_name='assets_prices_store')

# by default, it will display the first 5 rows of the store
assets_prices.head()

Price
Asset_Code Date                  
BNP.PA     2020-03-10  100.553603
           2020-03-09   99.605505
           2020-03-08   98.393106
           2020-03-07   98.996166
           2020-03-06  100.594698

### Loading parquet

In [8]:
assets_store = session.read_parquet('assets_attributes.parquet', keys=['Asset_Code'], store_name='assets_store')
assets_store.head()

,Sector,Country,Haircut
Asset_Code,,,
BNP.PA,Financial Services,France,0.1
CA.PA,Consumer Defensive,France,0.1
AC.PA,Consumer Cyclical,France,0.1
ENGI.PA,Utilities,France,0.1
CAP.PA,Technology,France,0.1


### Loading csv via pandas before session loads pandas dataframe   
Being able to load pandas dataframe allows us flexibility of data manipulations using dataframe in the way we are used to.  
This also meant that existing programs using dataframe to consume data can easily be migrated over to Atoti.

In [9]:
loans_positions_df =pd.read_csv('loans_positions.csv')

In [10]:
loans_positions_store = session.read_pandas(loans_positions_df, keys=['Account'], store_name='loans_positions_store')

In [11]:
loans_positions_store.head()

,Cash_Out
Account,
Bezos & MacKenzie,16000000.0
Buffet,54000000.0
Daniel Ek,9000000.0
Gates & Melinda,17000000.0
Musk,9000000.0


### Joining datastore

In [12]:
asset_positions_store.join(assets_store, mapping={'Asset_Code' : 'Asset_Code'})

In [13]:
asset_positions_store.join(loans_positions_store, mapping={'Account' : 'Account'})

In [14]:
asset_positions_store.join(assets_prices, mapping={'Asset_Code' : 'Asset_Code'})

This is how the datastore references looked like:

<img src="http://data.atoti.io/notebooks/collateral-shortfall-monitoring/datastore_references.png" alt="Datastore references" style="zoom:30%;" />

### Cube creation

In [15]:
asset_positions_cube = session.create_cube(asset_positions_store, 'Collateral_Management')

### Quick analysis with cube.visualize

In [17]:
# we can perform drill-down to different hierarchies in a pivot table
asset_positions_cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

In [18]:
# we can look at the price.VALUE across Date, further split the charts by Asset_Code
asset_positions_cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

In [20]:
# give a meaningful title to the visualization. This helps to reconcile the objective of the visual and also could be the title of 
# the widget when visual is published
asset_positions_cube.visualize('times-series')

Install the Atoti JupyterLab extension to see this widget.

In [22]:
asset_positions_cube.visualize('explore-dataset-using-pivot-table')

Install the Atoti JupyterLab extension to see this widget.

### Cube structure
During cube creation, numeric values are automatically created as measures. Non numeric values are automatically translated to levels under hierarchy of the same name. This can be [configured](https://www.atoti.io/documentation/lib/atoti.html#atoti.session.Session.create_cube) differently.

In [23]:
m = asset_positions_cube.measures
h = asset_positions_cube.hierarchies
lvl = asset_positions_cube.levels

Before we proceed with the data aggregation aspects, let's inspect the hierarchies created

In [24]:
h

Hierarchies()

We are going to set the hierarchy *Date* as a slicing hierarchy. A slicing hierarchy will not aggregate the data on all its members.  
This means that we always view a subset of the cube by one date by default, which is usually what is needed.

In [25]:
h["Date"].slicing = True

In [26]:
m

Measures()

### Creating new measures  
From the data we have, we can derive the following:   
$Market Value = Price \times Quantity$  
$Collateral Value = Market Value \times (1 - Haircut)$   
  
The above measures are aggregated over the Account and Asset Code levels in order to compute the Collateral Shortfall at account level:  
$Collateral Shortfall = Collateral Value - Cash Out$   
Where Cash Out is also aggregated at account level           

In [27]:
m['Market Value'] = tt.agg.sum(m['Price.VALUE'] * m['Quantity.SUM'], scope=tt.scope.origin('Account', 'Asset_Code'))

In [28]:
m['Collateral Value'] = tt.agg.sum(m['Price.VALUE'] * m['Quantity.SUM'] * (1-m['Haircut.VALUE']), tt.scope.origin('Account', 'Asset_Code'))

In [29]:
m['Cash Out'] = tt.agg.sum(m['Cash_Out.VALUE'], tt.scope.origin('Account'))

In [30]:
m['Collateral Shortfall'] = m['Collateral Value'] - m['Cash Out']

In [31]:
m

Measures()

### Monitoring Collateral Shortfall  
We created a pivot table for with the Collateral Shortfall, Market Value, Cash Out and Collateral Value for Accounts.  
Negative Collateral Shortfall are highlighted in red. Feel free to click on the `>` to drill-down to other hierarchies such as Sector to account for the shortfall. 

In [33]:
asset_positions_cube.visualize('Collateral Shortfall')

Install the Atoti JupyterLab extension to see this widget.

In view of all accounts, using a Gauge chart will show us that we are not yet in shortfall and how far we are from it.   
The red marker shows the total Market value, which is the maximum threshold before shortfall will occur.

In [35]:
asset_positions_cube.visualize('total-cash-out')

Install the Atoti JupyterLab extension to see this widget.

### Total Cash out Bank wide  
  
We can use a Tree map to visualize the asset concentration. A well diversified portfolio will help to reduce the collateral risks.

In [37]:
asset_positions_cube.visualize('Asset_Concentration')

Install the Atoti JupyterLab extension to see this widget.

## Atoti UI and Dashboard creation
Until now, we have created a few visualizations. We can right-click on the visuals to publish them as widgets.
These widgets can then be used to build a dashboard.  

<img src="http://data.atoti.io/notebooks/collateral-shortfall-monitoring/collateral_dashboard.gif" alt="collateral_dashboard" style="zoom:40%;" />

In [38]:
from IPython.display import Markdown as md 
md("[" + session.url + "/#/dashboard/d13](" + session.url + "/#/dashboard/d13)")

[http://localhost:50628/#/dashboard/d13](http://localhost:50628/#/dashboard/d13)

Click on the URL above to view the dashboard that was prepared. We can use the quick filter to select an account for viewing.  
We can also do a right-click drillthrough to investigate the underlying data.  
  
To play with the UI and explore the data, [you can have a look at our UI documentation here](https://www.activeviam.com/activeui/documentation/index.html).

## Simulations
Now that we have basic monitoring on Collateral Shortfall, we can do some simulations in the data cube without having to duplicate or reload data.  
  
### Setup Haircut Simulation
We are going to perform a Haircut simulation where there is a consumption slowdown in the Consumer Cyclical sector in Italy and France.  
In this use case, we will replace the value of haircut for the mentioned sector and country to see its downstream impact on the Collaterals.

In [39]:
# we have to do a setup_simulation, where scenarios added to this simulation will follow the same pattern of simulation 
# E.g. same levels and method on measure, in this case, replace Haircut
# we set the base_scenario_name to "Current Market Condition" - this is the original set of data currently in the cube 
haircut_simulation = asset_positions_cube.setup_simulation(
    "Haircut Simulation", 
    per=[lvl["Sector"], lvl["Country"]], 
    replace=[m["Haircut.VALUE"]], 
    base_scenario_name = "Current Market Condition"
)
haircut_simulation

Simulation(_name='Haircut Simulation', _levels=[Level(_name='Sector', _column_name='Sector', _data_type='string', _hierarchy=Hierarchy(_name='Sector', _levels={'Sector': ...}, _dimension='Hierarchies', _slicing=False), _comparator=None), Level(_name='Country', _column_name='Country', _data_type='string', _hierarchy=Hierarchy(_name='Country', _levels={'Country': ...}, _dimension='Hierarchies', _slicing=False), _comparator=None)], _multiply=[], _replace=[NamedMeasure(_name='Haircut.VALUE', _folder=None, _formatter='DOUBLE[#,###.00]', _visible=True)], _add=[], _base_scenario='Current Market Condition')

In [40]:
# create a new scenario with a meaningful name
consumption_slowdown = haircut_simulation.scenarios['Consumption Slowdown']

In [41]:
# parameters in the sequence of levels defined in the "per" during setup_simulation 
consumption_slowdown += ("Consumer Cyclical", "Italy", 0.20, tt.simulation.Priority.CRITICAL) 
consumption_slowdown += ("Consumer Cyclical", "France", 0.15, tt.simulation.Priority.CRITICAL)

#### Analysing the impact of Consumption Slowdown  
  
We will create a chart showing Collateral Shortfall against Account, split by *Haircut Simulation* - the Measure Simulation we just created.  
We will be able to see that with a slowdown in Italy ad France, *Gates & Melinda* will go into shortfall and the shortfall amount for *Buffet* increased dramatically.   
The Consumer Cyclical  in Italy and France does not impact *Daniel EK*, *Musk* and *Niel*.

In [43]:
asset_positions_cube.visualize('Consumption Slowdown')

Install the Atoti JupyterLab extension to see this widget.

With a pivot table, we can easily see the amount of collateral generated by the *Consumer Cyclical* sector in Italy and France.

In [45]:
asset_positions_cube.visualize('Consumption Slowdown by Sector')

Install the Atoti JupyterLab extension to see this widget.

### Market Value Simulation
We are going to create a new set of simulation where technology sector goes into a slump and price value for the sector dipped by 30%.  
We see that while the levels we will be simulating are the same as the previous simulation, now instead of replacing the Haircut value, we will multiply the Price value by 0.7.  

In [46]:
market_value_simulation = asset_positions_cube.setup_simulation("Market Value Simulation", per=[lvl["Sector"],lvl["Country"]], multiply=[m["Price.VALUE"]], base_scenario_name = "Current Market Condition")

In [47]:
tech_bubble = market_value_simulation.scenarios["Tech Bubble ?"]

In [48]:
# we are going to simulate a slump technology in all countries, therefore we pass the value `*` for country level
tech_bubble += ("Technology", "*", 0.70, tt.simulation.Priority.CRITICAL)

#### Analysing the impact of Technology slump  
  
As before, we plot a chart of Collateral Shortfall against Account. However, we are going to do a split by the *Market Simulation* that we just created.  
We will see the technology giant such as *Gates & Melinda* suffering greatly as their shortfall goes beyond 1.5 million.  
Likewise Buffet with 1/6 of assets in Technology also suffered.

In [50]:
asset_positions_cube.visualize("Tech Bubble ?")

Install the Atoti JupyterLab extension to see this widget.

As we drill-in further to simulation for *Buffet* vs *Gates*, we see that *Buffet* has greater shortfall as they have a higher Cash Out value for the Technology sector than *Gates* while their market value was the same. 

In [52]:
asset_positions_cube.visualize("Market Simulation - Gates vs Buffet")

Install the Atoti JupyterLab extension to see this widget.

### Setup Cash out Simulation

The previous simulations we have done are called Measure simulations.  
There is a different kind of simulation known as Source simulation.  

In Measure simulations, we modify the value of the measures in scenarios of the simulations without duplicating data.  
Source simulation on the other hand, is a simulation created by loading a new source of modified data to the cube.  

Let's modify the loans_positions_df that was previously loaded into the cube, and increase its Cash_Out value by 10%.  
For our *Cash out simulation*, we shall load this modified dataframe directly into the store as a scenario.

In [53]:
loans_positions_df["Cash_Out"] = loans_positions_df["Cash_Out"].apply(lambda x: (1+0.1)*x)

In [54]:
loans_positions_df

,Account,Cash_Out
0,Bezos & MacKenzie,17600000.0
1,Buffet,59400000.0
2,Daniel Ek,9900000.0
3,Gates & Melinda,18700000.0
4,Musk,9900000.0
5,Niel,15400000.0


In [55]:
# In Source Simulation, we do not perform simulation_setup. We just load it to the store as scenarios.
predictive_simulation = loans_positions_store.scenarios["Cash out Increase"].load_pandas(loans_positions_df, truncate=True)

#### Analysing the impact of increase in Cash Out

While cash out increases but collateral value remains the same, we see that the shortfall situation is worsen.

In [57]:
asset_positions_cube.visualize("Cash out Increase")

Install the Atoti JupyterLab extension to see this widget.

#### Activate Source Simulation

In Atoti UI, we have 2 kinds of widgets for simulations:  

- Measure Simulations
  - enable user to update, create new or duplicate simulations
  - available upon setup_simulation
- Source Simulations
  - enable users to upload their modified source files via UI as simulation
  - have to explicitly enable to be accessible from UI
  
Next we will enable source simulation.

In [58]:
loans_positions_store.source_simulation_enabled = True

Once we have enabled the source simulation, we can proceed to integrate the Source Simulations widget in our dashboard.  
<img src="http://data.atoti.io/notebooks/collateral-shortfall-monitoring/source_simulation.gif" alt="source_simulation" style="zoom:40%;" />  
  
To play with the source simulation, click on the link on the next output cell.

In [59]:
md("[" + session.url + "/#/dashboard/82b](" + session.url + "/#/dashboard/82b)")

[http://localhost:50628/#/dashboard/82b](http://localhost:50628/#/dashboard/82b)

In [60]:
# session.url -- this command gives the URL to the Atoti UI for the session
md("[" + session.url + "]("+ session.url + ")")

[http://localhost:50628](http://localhost:50628)